In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sn 
import numpy as np 
from keras.models import load_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df_full_feature = pd.read_csv('../Dataset/Normalisasi/processed_dataset_slice_norm.csv',sep=',')

In [3]:
lstm_full_model = load_model('lstm_slice.h5')
lstm_full_model

In [4]:
# Dataset Full Feature
full_feature_x = df_full_feature[df_full_feature.columns[:4]]
full_feature_y = df_full_feature['label']

full_feature_x.head(5)

,dst,src,Protocol,bytecount
0,0.647059,0.055556,0.0,0.000640
1,0.705882,0.333333,0.5,0.081254
2,0.882353,0.555556,0.0,0.000534
3,0.941176,0.833333,0.5,0.014585
4,0.647059,0.111111,1.0,0.618070


In [23]:
x_train,x_test,y_train,full_feature_y = train_test_split(full_feature_x,full_feature_y,test_size=0.30,random_state=9)

x_train_awal = x_train
x_test_awal = x_test
y_train_awal = y_train
y_test_awal = y_test

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape

(50540, 4, 1)

In [25]:
lstm_full_model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1580/1580 [==============================] - 15s 9ms/step - loss: 0.2258 - accuracy: 0.8779 - mean_squared_error: 0.4130 - mean_absolute_error: 0.4636
Epoch 2/10
1580/1580 [==============================] - 14s 9ms/step - loss: 0.2261 - accuracy: 0.8790 - mean_squared_error: 0.4131 - mean_absolute_error: 0.4636: 3s - loss: 0.2256 - accuracy: 0.8788 
Epoch 3/10
1580/1580 [==============================] - 14s 9ms/step - loss: 0.2245 - accuracy: 0.8801 - mean_squared_error: 0.4131 - mean_absolute_error: 0.4636
Epoch 4/10
1580/1580 [==============================] - 14s 9ms/step - loss: 0.2264 - accuracy: 0.8766 - mean_squared_error: 0.4132 - mean_absolute_error: 0.4636
Epoch 5/10
1580/1580 [==============================] - 14s 9ms/step - loss: 0.2293 - accuracy: 0.8779 - mean_squared_error: 0.4121 - mean_absolute_error: 0.4636
Epoch 6/10
1580/1580 [==============================] - 13s 8ms/step - loss: 0.2224 - accuracy: 0.8815 - mean_squared_error: 0.4136 - mean_absolute_err

In [26]:
y_pred_train = lstm_full_model.predict(x_train)
y_pred_train

array([[9.9999380e-01, 6.2281715e-06, 2.0753749e-23],
       [9.8934913e-01, 1.0650852e-02, 4.5778092e-16],
       [1.5719733e-07, 9.9999988e-01, 3.0682618e-33],
       ...,
       [1.3170725e-04, 9.9986827e-01, 1.7323264e-26],
       [1.5188462e-01, 8.4811544e-01, 4.6311869e-15],
       [1.2685916e-04, 9.9987316e-01, 5.2905631e-27]], dtype=float32)

In [21]:
def calculate_precision(TP, FP):
    return TP/(TP+FP)


def calculate_fnr(TP, FP, FN):
    return FN/(TP+FP)
    

In [30]:
y_pred_train = [np.argmax(element) for element in y_pred_train]
cm = confusion_matrix(y_train, y_pred_train, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)


[[16699  3052]
 [ 2712 28077]]
              precision    recall  f1-score   support

           0       0.90      0.91      0.91     30789
           1       0.86      0.85      0.85     19751

    accuracy                           0.89     50540
   macro avg       0.88      0.88      0.88     50540
weighted avg       0.89      0.89      0.89     50540

Accuracy: 88.5951721408785
Precision 86.02854051826284
False Negative Rate 15.723043635052289


In [32]:
y_pred_val = lstm_full_model.predict(x_test)
y_pred_val

array([[4.3321484e-01, 5.6678516e-01, 6.1818911e-10],
       [9.6160442e-01, 3.8395595e-02, 7.3492094e-11],
       [2.7360086e-06, 9.9999726e-01, 1.6576646e-31],
       ...,
       [9.9999475e-01, 5.2965388e-06, 7.1553007e-35],
       [3.2870933e-06, 9.9999666e-01, 2.3123538e-28],
       [9.9999428e-01, 5.7800066e-06, 8.5276490e-35]], dtype=float32)

In [33]:
y_pred_val = [np.argmax(element) for element in y_pred_val]
cm = confusion_matrix(y_test, y_pred_val, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_test, y_pred_val))
print("Accuracy:",accuracy_score(y_test, y_pred_val)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)

[[ 7049  1382]
 [ 1241 11989]]
              precision    recall  f1-score   support

           0       0.90      0.91      0.90     13230
           1       0.85      0.84      0.84      8431

    accuracy                           0.88     21661
   macro avg       0.87      0.87      0.87     21661
weighted avg       0.88      0.88      0.88     21661

Accuracy: 87.89067910068788
Precision 85.0301568154403
False Negative Rate 16.670687575392037


## Testing Dataset

In [5]:
test_full_feature = pd.read_csv('../Dataset/Normalisasi/test_processed_dataset_slice_norm.csv',sep=',')

In [9]:
# Dataset Full Feature
full_feature_x = df_full_feature[df_full_feature.columns[:4]]
full_feature_y = df_full_feature['label']

full_feature_x.head(5)

,dst,src,Protocol,bytecount
0,0.647059,0.055556,0.0,0.000640
1,0.705882,0.333333,0.5,0.081254
2,0.882353,0.555556,0.0,0.000534
3,0.941176,0.833333,0.5,0.014585
4,0.647059,0.111111,1.0,0.618070


In [14]:
full_feature_x = full_feature_x.to_numpy()
full_feature_x = np.reshape(full_feature_x, (full_feature_x.shape[0], full_feature_x.shape[1], 1))
full_feature_x

array([[[6.47058824e-01],
        [5.55555556e-02],
        [0.00000000e+00],
        [6.40109284e-04]],

       [[7.05882353e-01],
        [3.33333333e-01],
        [5.00000000e-01],
        [8.12536148e-02]],

       [[8.82352941e-01],
        [5.55555556e-01],
        [0.00000000e+00],
        [5.33535418e-04]],

       ...,

       [[8.82352941e-01],
        [1.66666667e-01],
        [0.00000000e+00],
        [1.03909520e-04]],

       [[7.64705882e-01],
        [9.44444444e-01],
        [1.00000000e+00],
        [2.82570275e-01]],

       [[9.41176471e-01],
        [7.22222222e-01],
        [1.00000000e+00],
        [6.21212636e-01]]])

In [18]:
test_pred = lstm_full_model.predict(full_feature_x)
test_pred

array([[9.9999857e-01, 1.4143640e-06, 3.5144482e-33],
       [8.5350327e-02, 9.1464967e-01, 2.7224068e-19],
       [9.9999571e-01, 4.3207697e-06, 2.9391574e-33],
       ...,
       [9.9999201e-01, 8.0182317e-06, 7.7147588e-32],
       [7.3427802e-01, 2.6572204e-01, 2.4470814e-12],
       [9.9944252e-01, 5.5748888e-04, 1.0543759e-19]], dtype=float32)

In [22]:
pred_test = [np.argmax(element) for element in test_pred]
cm = confusion_matrix(full_feature_y, pred_test, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(full_feature_y, pred_test))
print("Accuracy:",accuracy_score(full_feature_y, pred_test)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)

[[25000  3182]
 [ 5068 38951]]
              precision    recall  f1-score   support

           0       0.92      0.88      0.90     44019
           1       0.83      0.89      0.86     28182

    accuracy                           0.89     72201
   macro avg       0.88      0.89      0.88     72201
weighted avg       0.89      0.89      0.89     72201

Accuracy: 88.57356546308223
Precision 83.14487162431821
False Negative Rate 10.582679260343223
